In [2]:
import pandas as pd

In [3]:
#Importando dados:
melbourn_file = pd.read_csv('../data/melbourn_housing_snaps/melb_data.csv')
melbourn_file.dropna(axis=0, inplace=True)

In [5]:
melbourn_file.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [12]:
#Separa um target de previsão:

y = melbourn_file.Price

In [10]:
#Escolhendo as features
features = ['Rooms','Bathroom','Landsize','Lattitude','Longtitude']
X =  melbourn_file[features]

In [18]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [17]:
#Treinando o modelo:

from sklearn.tree import DecisionTreeRegressor

model_home = DecisionTreeRegressor(random_state=1)
model_home.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [23]:
#Comparando modelo previsto com Real:

predictions = model_home.predict(X)
print("Prediction:", predictions[:5])
print("Target:", y[:5].values)

Prediction: [1035000. 1465000. 1600000. 1876000. 1636000.]
Target: [1035000. 1465000. 1600000. 1876000. 1636000.]


In [25]:
#### Validação de modelo: ####

# MEAN ABSOLUTE ERROR (MAE):
# Error= actual - predicted

from sklearn.metrics import mean_absolute_error

err = mean_absolute_error(y, predictions)
err

1115.7467183128902

In [33]:
#Validação para novos dados:

from sklearn.model_selection import train_test_split

train_X, val_X, train_Y, val_Y = train_test_split(X, y, random_state=1)


In [38]:
#Verificar como foi feita a separação:

train_X.shape, val_X.shape, X.shape

# train_X: FEATURES TREINAMENTOS
# train_Y: VALORES DAS FEATURES DE TREINAMENTOS
# val_X: NOVAS FEATURES DE VALIDAÇÃO

# val_Y: VALORES DA FEATURE DE VALIDAÇÃO PARA COMPARAR COM AS PREDIÇÕES REALIZADAS.

((4647, 5), (1549, 5), (6196, 5))

In [34]:
melbourn_model = DecisionTreeRegressor()
melbourn_model.fit(train_X, train_Y)

val_predictions = melbourn_model.predict(val_X)
print(mean_absolute_error(val_Y, val_predictions))

255247.21626856035


In [47]:
#Underfitting e Overfitting:

#parametro max_leaf_nodes > determina o número máximo de nós que o modelo deve ter para ele não ficar ruim.

In [52]:
#Função para achar a melhor arvore:

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model_ = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model_.fit(train_X,train_y)
    preds_val = model_.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return (mae)

In [56]:
# Testando a melhor leaf node:

erro = 1e10

for max_leaf_nodes in [5,50, 100, 250, 500, 1000, 5000]:
    error_node = get_mae(max_leaf_nodes, train_X,val_X, train_Y, val_Y)
    if error_node < erro:
        erro = error_node
        best_leaf_nodes = max_leaf_nodes
print(f"A melhor leaf node é {best_leaf_nodes} - com erro de {error_node}")



A melhor leaf node é 250 - com erro de 257788.11878631375


In [59]:
model_home = DecisionTreeRegressor(max_leaf_nodes=best_leaf_nodes, random_state=1)

model_home.fit(X,y)
prediction = model_home.predict(X)

In [60]:
display(prediction)

array([1196298.17307692, 1254673.91304348, 1609800.        , ...,
        393070.45454545,  594064.11057692, 2677500.        ])